In [4]:
import gym
from gym.spaces import Discrete
import torch.nn as nn
import torch.optim as optim
import numpy as np
from SocketPrototypes.MCS_Device_Interface import MCS_Device_Interface

In [ ]:
class OpenAIGymAPI:
    
    def __init__(self):
        self.env = 'CartPole-v1'
        self.state = None
        self.mcs_interface = MCS_Device_Interface()

    def initialize_training(self):
        self.state = self.env.reset()
        self.total_reward = 0
        return self.state
    
    def run_episodes(self, num_episodes):
        rewards = []

        for episode in range(num_episodes):
            print(f"Running episode {episode + 1}...")
            episode_reward = self.run_episode()
            rewards.append(episode_reward)
        print(f"Finished running {num_episodes} episodes.")
        return rewards

    def run_episode(self):
        self.state = self.initialize_training()
        terminated = False

        while not terminated:
            pole_angle = self.state[2]
            self.mcs_interface.send_pole_angle(pole_angle)
            action = self.mcs_interface.receive_neuron_action()
            next_state, reward, terminated, info = self.env.step(action)
            
            if terminated:
                self.mcs_interface.stimulate_neurons(reward)

            self.state = next_state
            self.total_reward += reward
        
        print(f"Episode finished with total reward: {self.total_reward}")
        return self.total_reward
    
    def run_single_frame(self, timestep_data):
        pole_angle = self.state[2]
        pole_angular_velocty = self.state[3]
        self.mcs_interface.send_pole_angle(pole_angle)
        action = self.mcs_interface.receive_neuron_action(self, time, timestep_data)
        observation, reward, terminated, _,  _, _, _ = self.env.step(action)
        return observation[2], observation[3], reward, terminated